In [53]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm


In [54]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
n_residual_blocks = 5
# The data, split between train and test sets
(x, _), (y, _) = keras.datasets.mnist.load_data()
# Concatenate all of the images together
data = np.concatenate((x, y), axis=0)
# Round all pixel values less than 33% of the max 256 value to 0
# anything above this value gets rounded up to 1 so that all values are either
# 0 or 1
data = np.where(data < (0.33 * 256), 0, 1)
data = data.astype(np.float32)


In [55]:
data.shape

(70000, 28, 28)

In [56]:
# The first layer is the PixelCNN layer. This layer simply
# builds on the 2D convolutional layer, but includes masking.
class PixelConvLayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super(PixelConvLayer, self).__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)

    def build(self, input_shape):
        # Build the conv2d layer to initialize kerneconv2dl variables
        self.conv.build(input_shape)
        # Use the initialized kernel to create the mask
        kernel_shape = self.conv.kernel.get_shape()
        self.mask = np.zeros(shape=kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, : kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == "B":
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0

    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return self.conv(inputs)


# Next, we build our residual block layer.
# This is just a normal residual block, but based on the PixelConvLayer.
class ResidualBlock(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )
        self.pixel_conv = PixelConvLayer(
            mask_type="B",
            filters=filters // 2,
            kernel_size=3,
            activation="relu",
            padding="same",
        )
        self.conv2 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return keras.layers.add([inputs, x])


In [57]:
inputs = keras.Input(shape=input_shape)
x = PixelConvLayer(
    mask_type="A", filters=128, kernel_size=7, activation="relu", padding="same"
)(inputs)

for _ in range(n_residual_blocks):
    x = ResidualBlock(filters=128)(x)

for _ in range(2):
    x = PixelConvLayer(
        mask_type="B",
        filters=128,
        kernel_size=1,
        strides=1,
        activation="relu",
        padding="valid",
    )(x)

out = keras.layers.Conv2D(
    filters=1, kernel_size=1, strides=1, activation="sigmoid", padding="valid"
)(x)

pixel_cnn = keras.Model(inputs, out)
adam = keras.optimizers.Adam(learning_rate=0.0005)
pixel_cnn.compile(optimizer=adam, loss="binary_crossentropy" )

pixel_cnn.summary()
pixel_cnn.fit(
    x=data, y=data, batch_size=128, epochs=50, validation_split=0.1, verbose=2
)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
pixel_conv_layer_32 (PixelCo (None, 28, 28, 128)       6400      
_________________________________________________________________
residual_block_20 (ResidualB (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_21 (ResidualB (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_22 (ResidualB (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_23 (ResidualB (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_24 (ResidualB (None, 28, 28, 128)       9862

In [84]:
 tf.random.uniform((4,))

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.9412769 , 0.01695979, 0.82117116, 0.6151278 ], dtype=float32)>

In [86]:
tf.math.ceil([-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.0])

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([-1., -1., -0.,  1.,  2.,  2.,  2.], dtype=float32)>

In [85]:
from IPython.display import Image, display

# Create an empty array of pixels.
batch = 4
pixels = np.zeros(shape=(batch,) + (pixel_cnn.input_shape)[1:])
batch, rows, cols, channels = pixels.shape

# Iterate over the pixels because generation has to be done sequentially pixel by pixel.
for row in tqdm(range(rows)):
    for col in range(cols):
        for channel in range(channels):
            # Feed the whole array and retrieving the pixel value probabilities for the next
            # pixel.
            probs = pixel_cnn.predict(pixels)[:, row, col, channel]
            
            # Use the probabilities to pick pixel values and append the values to the image
            # frame.
            
            print(probs)
            #print(probs - tf.random.uniform(probs.shape))
            #print(tf.math.ceil(probs - tf.random.uniform(probs.shape , seed = 10)))
            pixels[:, row, col, channel] = tf.math.ceil(
                probs - tf.random.uniform(probs.shape)
            )


def deprocess_image(x):
    # Stack the single channeled black and white image to RGB values.
    x = np.stack((x, x, x), 2)
    # Undo preprocessing
    x *= 255.0
    # Convert to uint8 and clip to the valid range [0, 255]
    x = np.clip(x, 0, 255).astype("uint8")
    return x


# Iterate over the generated images and plot them with matplotlib.
for i, pic in enumerate(pixels):
    keras.preprocessing.image.save_img(
        "generated_image_{}.png".format(i), deprocess_image(np.squeeze(pic, -1))
    )

display(Image("generated_image_0.png"))
display(Image("generated_image_1.png"))
display(Image("generated_image_2.png"))
display(Image("generated_image_3.png"))


  0%|          | 0/28 [00:00<?, ?it/s]

[2.345803e-07 2.345803e-07 2.345803e-07 2.345803e-07]
[3.923414e-07 3.923414e-07 3.923414e-07 3.923414e-07]
[3.0153237e-07 3.0153237e-07 3.0153237e-07 3.0153237e-07]
[1.0634633e-06 1.0634633e-06 1.0634633e-06 1.0634633e-06]
[1.9531935e-06 1.9531935e-06 1.9531935e-06 1.9531935e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.2067177e-06 2.2067177e-06 2.2067177e-06]
[2.2067177e-06 2.20671

  4%|▎         | 1/28 [00:01<00:28,  1.06s/it]

[2.2067136e-06 2.2067136e-06 2.2067136e-06 2.2067136e-06]
[2.2067093e-06 2.2067093e-06 2.2067093e-06 2.2067093e-06]
[2.2067159e-06 2.2067159e-06 2.2067159e-06 2.2067159e-06]
[2.2067159e-06 2.2067159e-06 2.2067159e-06 2.2067159e-06]
[7.01618e-08 7.01618e-08 7.01618e-08 7.01618e-08]
[3.4926475e-06 3.4926475e-06 3.4926475e-06 3.4926475e-06]
[7.573923e-07 7.573923e-07 7.573923e-07 7.573923e-07]
[9.98258e-06 9.98258e-06 9.98258e-06 9.98258e-06]
[3.1216587e-05 3.1216587e-05 3.1216587e-05 3.1216587e-05]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00022052]
[0.00022052 0.00022052 0.00022052 0.00

  7%|▋         | 2/28 [00:02<00:26,  1.02s/it]

...

[0.00693341 0.00063893 0.00693341 0.00019656]
[0.001551  0.001551  0.001551  0.0002829]
[0.00033534 0.00033534 0.00033534 0.00020894]
[6.756427e-05 6.756427e-05 6.756427e-05 6.756427e-05]
[1.1216427e-05 1.1216427e-05 1.1216427e-05 1.1216427e-05]
[3.5795672e-06 4.3520117e-07 3.5795672e-06 3.5795606e-06]
[4.8436690e-05 1.1656412e-06 4.8436690e-05 4.8436785e-05]
[2.8492184e-04 8.2247689e-06 3.2705069e-04 2.8492158e-04]
[5.9211679e-04 6.2816260e-05 6.2015274e-04 1.3055902e-02]
[1.3517849e-03 2.3973525e-05 3.7162495e-03 1.1225772e-01]
[0.02057226 0.00393649 0.01144963 0.5492337 ]
[0.02057226 0.06019994 0.01015718 0.97882825]
[0.02057226 0.5958819  0.01069267 0.9886386 ]
[0.02057226 0.9998122  0.00844428 0.38796598]
[0.02057226 0.999503   0.00761438 0.00443534]
[0.02057226 0.99818194 0.00491395 0.00446362]
[0.02057226 0.8085781  0.00484886 0.00641202]
[0.02057226 0.31542283 0.00396817 0.01366487]
[0.02057226 0.13993827 0.00225599 0.02204752]
[0.02057226 0.60181874 0.00160335 0.02996018]
[0.0

 96%|█████████▋| 27/28 [00:27<00:01,  1.00s/it]

[0.001551   0.001551   0.8183025  0.00170328]
[3.3533669e-04 3.3533669e-04 3.8472760e-01 3.8205838e-04]
[6.7564273e-05 6.7564273e-05 2.0196147e-01 5.1034709e-05]
[1.1216427e-05 1.1216427e-05 5.6224518e-02 1.1216427e-05]
[3.5795672e-06 4.3520032e-07 3.5795672e-06 3.5795638e-06]
[4.8436690e-05 1.1656412e-06 4.8436690e-05 4.8436832e-05]
[2.8492184e-04 8.2247852e-06 2.8492184e-04 8.7498745e-05]
[5.9211679e-04 6.2816260e-05 9.0983848e-04 3.1804014e-04]
[1.3517849e-03 2.1003716e-05 1.8940388e-03 1.5853620e-03]
[0.02057226 0.00310324 0.0080223  0.4867739 ]
[0.02057226 0.05564062 0.01069513 0.9740975 ]
[0.02057226 0.6577685  0.01159313 0.97604126]
[0.02057226 0.9998379  0.00715889 0.19957528]
[0.02057226 0.99931395 0.00897493 0.00207641]
[0.02057226 0.9987871  0.00748175 0.00252826]
[0.02057226 0.99850035 0.00963451 0.00396231]
[0.02057226 0.99728906 0.00525892 0.00844421]
[0.02057226 0.9679179  0.00513797 0.01421759]
[0.02057226 0.6922874  0.0039757  0.02335427]
[0.02057226 0.33808136 0.00518

100%|██████████| 28/28 [00:28<00:00,  1.01s/it]

[0.02454364 0.01287531 0.9973212  0.7018476 ]
[0.00409542 0.00249778 0.9985014  0.48433676]
[1.8399636e-03 9.6208876e-04 9.8346901e-01 3.3574796e-03]
[2.5902016e-04 2.7095355e-04 8.9240301e-01 3.5131534e-04]
[9.208640e-05 9.918810e-05 6.393941e-01 3.311634e-05]
[1.4236598e-05 2.4702143e-05 2.6195969e-02 6.6095486e-06]


In [31]:
arr.shape


(4, 28, 28, 1)

In [6]:
mask = np.zeros((12,12))

In [8]:
mask.shape

(12, 12)

In [10]:

# importing numpy
import numpy as np
  
array = np.zeros((10,10, 2))
array[: array.shape[0] // 2, ] = 1.0

In [11]:
array.shape

(10, 10, 2)

In [ ]:
mask[: mask.shape[0] // 2] = 1.0

In [ ]:
mask

In [ ]:
filter_size = mask.shape[0]
filter_center = filter_size / 2

# Zero out all weights below the center.
mask[math.ceil(filter_center):] = 0

In [ ]:
mask